# Everything from Part2 of the assignment

In [2]:
import geocoder
import pandas as pd
data = pd.read_csv("neighborhoods.csv")
postcode = pd.read_csv("Geospatial_Coordinates.csv")
lat = []
lon = []
for i in range(len(data['PostalCode'])):
    for j in range(len(postcode['Postal Code'])):
        if data.loc[i,'PostalCode'] == postcode.loc[j,'Postal Code']:
            lat.append(postcode.loc[j,'Latitude'])
            lon.append(postcode.loc[j,'Longitude'])
data['Latitude'] = lat
data['Longitude'] = lon
data = data.drop('Unnamed: 0',axis = 1)

## Checking the dadaframe before going forward

In [4]:
data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


## Everything looks good so we can start clustering now.

# 1. Import the necessary modules

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.....')

Libraries imported.....


# 2. Use geopy library to get the latitude and longitude values of Toronto City.

In [7]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


# 3. Create a map of Toronto with neighborhoods superimposed on top.

In [10]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood,code in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood'], data['PostalCode']):
    label = '{},Borough : {}, Postal-Code :{}'.format(neighborhood, borough, code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# 4. Now that it is done we can start using the Foursquare API 

In [68]:
CLIENT_ID = '*********' # your Foursquare ID
CLIENT_SECRET = '*****' # your Foursquare Secret
VERSION = '***********' # Version                  #deleted because contains sensitive data
LIMIT = 100

# 5. Define the function to extract the category from the venues json data

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# 6. Time to explore the neighborhoods in Toronto

In [35]:
# Here we shall restrict our exploration within 500 meters radius of the neiughborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    count = 0
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        count+=1
        print(count,name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# 7. Create the Dataframe that has all the data about the Neighborhoods and the venues within the 500 metres radius of the Neighborhood.

In [36]:
toronto_venues = getNearbyVenues(names = data['Neighborhood'], 
                                   latitudes = data['Latitude'], 
                                   longitudes = data['Longitude'])

1 Parkwoods
2 Victoria Village
3 Harbourfront,Regent Park
4 Lawrence Heights,Lawrence Manor
5 Queen's Park
6 Islington Avenue
7 Rouge,Malvern
8 Don Mills North
9 Woodbine Gardens,Parkview Hill
10 Ryerson,Garden District
11 Glencairn
12 Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
13 Highland Creek,Rouge Hill,Port Union
14 Flemingdon Park,Don Mills South
15 Woodbine Heights
16 St. James Town
17 Humewood-Cedarvale
18 Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
19 Guildwood,Morningside,West Hill
20 The Beaches
21 Berczy Park
22 Caledonia-Fairbanks
23 Woburn
24 Leaside
25 Central Bay Street
26 Christie
27 Cedarbrae
28 Hillcrest Village
29 Bathurst Manor,Downsview North,Wilson Heights
30 Thorncliffe Park
31 Adelaide,King,Richmond
32 Dovercourt Village,Dufferin
33 Scarborough Village
34 Fairview,Henry Farm,Oriole
35 Northwood Park,York University
36 East Toronto
37 Harbourfront East,Toronto Islands,Union Station
38 Little Portugal,Trinity
39 East Birchmou

In [37]:
toronto_venues.head(15)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,"Harbourfront,Regent Park",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Harbourfront,Regent Park",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Harbourfront,Regent Park",43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center


#### Let's have a look at the number of venues we got

In [38]:
print("We have {} venues near all {} neighborhoods in Toronto".format(toronto_venues.shape[0],len(toronto_venues['Neighborhood'].unique())))

We have 2231 venues near all 100 neighborhoods in Toronto


An important observation to make here is that although we have processed all the 103 neighborhoods we have data for 100 neighborhoods which means that foursquare has no data on venues within 500 meters radius of 3 neighborhoods.

#### Let's figure out which Neighborhoods are missing

In [40]:
for i in data['Neighborhood']:
    if i not in toronto_venues['Neighborhood'].unique():
        print(i)

Islington Avenue
Newtonbrook,Willowdale
Upper Rouge


### Now that we have the missing Neighborhoods let's take one of them and check our observation.    

Let's take ***Islington Avenue*** as the target Neighborhood and check the co-ordinates (43.667856,-79.532242) in **Foursquare**.

In [41]:
url = 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION,43.667856,-79.532242,100,500)

In [42]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c285b06db04f53ab4fc0fad'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Edenbridge - Humber Valley',
  'headerFullLocation': 'Edenbridge - Humber Valley, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.6723560045, 'lng': -79.52603259414254},
   'sw': {'lat': 43.6633559955, 'lng': -79.53845140585746}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

### The warning looks worth noticing since the 'items' is empty.

In [45]:
results['response']['warning']['text']

"There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."

# 8. Let's expand our search area to 1.5 km.

In [46]:
toronto_venues = getNearbyVenues(names = data['Neighborhood'], 
                                   latitudes = data['Latitude'], 
                                   longitudes = data['Longitude'], radius=1500)

1 Parkwoods
2 Victoria Village
3 Harbourfront,Regent Park
4 Lawrence Heights,Lawrence Manor
5 Queen's Park
6 Islington Avenue
7 Rouge,Malvern
8 Don Mills North
9 Woodbine Gardens,Parkview Hill
10 Ryerson,Garden District
11 Glencairn
12 Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
13 Highland Creek,Rouge Hill,Port Union
14 Flemingdon Park,Don Mills South
15 Woodbine Heights
16 St. James Town
17 Humewood-Cedarvale
18 Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
19 Guildwood,Morningside,West Hill
20 The Beaches
21 Berczy Park
22 Caledonia-Fairbanks
23 Woburn
24 Leaside
25 Central Bay Street
26 Christie
27 Cedarbrae
28 Hillcrest Village
29 Bathurst Manor,Downsview North,Wilson Heights
30 Thorncliffe Park
31 Adelaide,King,Richmond
32 Dovercourt Village,Dufferin
33 Scarborough Village
34 Fairview,Henry Farm,Oriole
35 Northwood Park,York University
36 East Toronto
37 Harbourfront East,Toronto Islands,Union Station
38 Little Portugal,Trinity
39 East Birchmou

In [47]:
toronto_venues.head(15)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
4,Parkwoods,43.753259,-79.329656,A&W Canada,43.760643,-79.326865,Fast Food Restaurant
5,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
6,Parkwoods,43.753259,-79.329656,Tim Hortons,43.752814,-79.314067,Coffee Shop
7,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746086,-79.324978,Grocery Store
8,Parkwoods,43.753259,-79.329656,Shoppers Drug Mart,43.745303,-79.325249,Pharmacy
9,Parkwoods,43.753259,-79.329656,Food Basics,43.760865,-79.326015,Supermarket


#### Let's have a look at the number of venues we got this time

In [48]:
print("We have {} venues near all {} neighborhoods in Toronto".format(toronto_venues.shape[0],len(toronto_venues['Neighborhood'].unique())))

We have 6725 venues near all 103 neighborhoods in Toronto


### So we have got all the 103 Neighborhoods this time.

### Now let's check the number of venues per Neighborhood

In [49]:
# Number of venues per neighbourhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Adelaide,King,Richmond                                                                                                             100
Agincourt                                                                                                                           57
Agincourt North,L'Amoreaux East,Milliken,Steeles East                                                                               75
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown                              29
Alderwood,Long Branch                                                                                                               42
Bathurst Manor,Downsview North,Wilson Heights                                                                                       37
Bayview Village                                                                                                                     11
Bedford Park,Lawrence Manor East          

# 9. Analyze Each Neighborhood

In [51]:
print("Number of unique categories are : ",len(toronto_venues['Venue Category'].unique()))

Number of unique categories are :  334


In [56]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood_name'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape[1]) # checking whether all the 334 categories are included and total number of columns should be 335
toronto_onehot.head()

335


,Neighborhood_name,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,

### One problem that we faced during the craetion of this dataframe is that one of the 334 categories of the venues is called 'Neighborhood'. So we have to careful about giving a new name to the column. Here we have given the name 'Neighborhood_name'. Also the total number of column is 335 as expected.

In [57]:
#the shape of the dataframe is :
toronto_onehot.shape

(6725, 335)

This shape looks perfect!!

# 10. Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [58]:
toronto_grouped=toronto_onehot.groupby('Neighborhood_name').mean().reset_index()
toronto_grouped

,Neighborhood_name,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,

### So let's take a neighbourhood and print it's top 5 most common venues.

In [59]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood_name']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood_name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0        Hotel  0.06
1         Café  0.05
2  Coffee Shop  0.05
3   Restaurant  0.04
4      Theater  0.04


----Agincourt----
                 venue  freq
0   Chinese Restaurant  0.16
1          Coffee Shop  0.07
2        Shopping Mall  0.04
3                 Park  0.04
4  Japanese Restaurant  0.04


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0  Chinese Restaurant  0.17
1   Korean Restaurant  0.05
2              Bakery  0.05
3         Coffee Shop  0.05
4     Bubble Tea Shop  0.05


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0           Coffee Shop  0.17
1  Fast Food Restaurant  0.17
2         Grocery Store  0.10
3           Pizza Place  0.10
4                  Café  0.03


----Alderwood,Long Branch----
              venue  freq
0              Café  0.05
1  Toy / Game Store  0.05
2     Grocery 

4           Pizza Place  0.05


----East Toronto----
              venue  freq
0  Greek Restaurant  0.13
1              Café  0.06
2            Bakery  0.05
3       Pizza Place  0.05
4       Coffee Shop  0.05


----Emery,Humberlea----
            venue  freq
0     Golf Course  0.12
1     Coffee Shop  0.12
2      Beer Store  0.06
3  Hardware Store  0.06
4    Liquor Store  0.06


----Fairview,Henry Farm,Oriole----
                       venue  freq
0                Coffee Shop  0.12
1             Clothing Store  0.07
2                       Park  0.05
3       Fast Food Restaurant  0.05
4  Middle Eastern Restaurant  0.05


----First Canadian Place,Underground city----
                venue  freq
0               Hotel  0.07
1         Coffee Shop  0.05
2                Café  0.05
3             Theater  0.04
4  Italian Restaurant  0.04


----Flemingdon Park,Don Mills South----
                       venue  freq
0                Coffee Shop  0.08
1                        Gym  0.05
2        Ja

4  American Restaurant  0.03


----Scarborough Village----
                  venue  freq
0  Fast Food Restaurant  0.12
1        Sandwich Place  0.12
2              Pharmacy  0.09
3    Chinese Restaurant  0.06
4           Wings Joint  0.06


----Silver Hills,York Mills----
                    venue  freq
0             Coffee Shop  0.13
1                    Bank  0.05
2  Furniture / Home Store  0.05
3            Burger Joint  0.03
4                    Park  0.03


----St. James Town----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.07
2   Restaurant  0.05
3        Hotel  0.04
4    Gastropub  0.04


----Stn A PO Boxes 25 The Esplanade----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.07
2        Hotel  0.05
3   Restaurant  0.04
4         Park  0.04


----Studio District----
                   venue  freq
0            Coffee Shop  0.09
1                   Café  0.06
2                 Bakery  0.06
3  Vietnamese Restaurant  0.05
4                    Bar  0.05


--

# 11.  Let's put top ten venues of each neighborhood into a **Pandas** dataframe

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood_name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood_name'] = toronto_grouped['Neighborhood_name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Hotel,Coffee Shop,Café,Theater,Restaurant,Italian Restaurant,Japanese Restaurant,Gastropub,Concert Hall,Thai Restaurant
1,Agincourt,Chinese Restaurant,Coffee Shop,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Bakery,Breakfast Spot,Department Store,Park,Shopping Mall
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Chinese Restaurant,Korean Restaurant,Bakery,Coffee Shop,Bubble Tea Shop,Pharmacy,Pizza Place,BBQ Joint,Dessert Shop,Noodle House
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Fast Food Restaurant,Coffee Shop,Grocery Store,Pizza Place,Hardware Store,Bus Line,Café,Sandwich Place,Greek Restaurant,Beer Store
4,"Alderwood,Long Branch",Burger Joint,Toy / Game Store,Pizza Place,Pharmacy,Café,Grocery Store,Coffee Shop,Park,Beer Store,Bank
5,"Bathurst Manor,Downsview North,Wilson Heights",Park,Pizza Place,Coffee Shop,Eastern European Restaurant,Gift Shop,Fast Food Restaurant,Bank,Baseball Field,Gas Station,Sandwich Place
6,Bayview Village,Bank,Trail,Grocery Store,Japanese Restaurant,Park,Fast Food Restaurant,Café,Chinese Restaurant,Shopping Mall,Electronics Store
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Bakery,Sushi Restaurant,Bagel Shop,Pizza Place,Italian Restaurant,Asian Restaurant,Café,Tea Room,Sandwich Place
8,Berczy Park,Café,Coffee Shop,Hotel,Park,Restaurant,Japanese Restaurant,Seafood Restaurant,Bakery,Gastropub,Farmers Market
9,"Birch Cliff,Cliffside West",Park,Hotel,Skating Rink,Golf Course,General Entertainment,Gym,Gym Pool,Café,Diner,Bank


# 12. Cluster Neighborhood

In [64]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(len(kmeans.labels_))
kmeans.labels_

103


array([0, 4, 4, 1, 2, 2, 4, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0,
       0, 1, 4, 1, 2, 0, 0, 2, 0, 1, 0, 2, 2, 0, 1, 1, 2, 1, 0, 2, 2, 0,
       2, 2, 1, 1, 0, 0, 0, 0, 0, 4, 0, 2, 0, 2, 0, 1, 2, 4, 2, 2, 2, 0,
       1, 2, 0, 2, 2, 1, 1, 0, 1, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 0, 0, 2,
       0, 0, 1, 2, 2, 3, 1, 1, 2, 2, 1, 1, 2, 2, 2], dtype=int32)

In [66]:
toronto_merged = data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood_name'), on='Neighborhood')
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Coffee Shop,Pharmacy,Bank,Train Station,Golf Course,BBQ Joint,Discount Store,Sandwich Place,Caribbean Restaurant,Pizza Place
1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Coffee Shop,Middle Eastern Restaurant,Fast Food Restaurant,Gym,Grocery Store,Portuguese Restaurant,Cosmetics Shop,Beer Store,Spa,Soccer Field
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,4,Coffee Shop,Café,Breakfast Spot,Italian Restaurant,Restaurant,Park,Theater,Bakery,Bar,Farmers Market
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Sandwich Place,Cosmetics Shop,Furniture / Home Store,Dessert Shop,Vietnamese Restaurant,Bakery,Sporting Goods Shop
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Japanese Restaurant,Gastropub,Park,Tea Room,Ramen Restaurant,Pizza Place,Italian Restaurant,Bookstore,Bubble Tea Shop
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,2,Shopping Mall,Pharmacy,Park,Café,Grocery Store,Bus Stop,Camera Store,Bus Line,Supermarket,Golf Course
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,4,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Department Store,Cosmetics Shop,Fruit & Vegetable Store,Bus Station,Liquor Store,Spa,Paper / Office Supplies Store
7,M3B,North York,Don Mills North,43.745906,-79.352188,2,Japanese Restaurant,Coffee Shop,Restaurant,Burger Joint,Park,Pizza Place,Bank,Supermarket,Asian Restaurant,Café
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,0,Fast Food Restaurant,Pizza Place,Pharmacy,Brewery,Athletics & Sports,Skating Rink,Intersection,Park,Office,Bakery
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Gastropub,Tea Room,Café,Japanese Restaurant,Cosmetics Shop,Italian Restaurant,American Restaurant,Restaurant,Ramen Restaurant


In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 13. Examine the clusters

### Cluster 1

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,0,Coffee Shop,Pharmacy,Bank,Train Station,Golf Course,BBQ Joint,Discount Store,Sandwich Place,Caribbean Restaurant,Pizza Place
8,"Woodbine Gardens,Parkview Hill",0,Fast Food Restaurant,Pizza Place,Pharmacy,Brewery,Athletics & Sports,Skating Rink,Intersection,Park,Office,Bakery
9,"Ryerson,Garden District",0,Coffee Shop,Gastropub,Tea Room,Café,Japanese Restaurant,Cosmetics Shop,Italian Restaurant,American Restaurant,Restaurant,Ramen Restaurant
11,"Cloverdale,Islington,Martin Grove,Princess Gar...",0,Convenience Store,Hotel,Bank,Park,Sandwich Place,Skating Rink,Gym,Gym / Fitness Center,Fish & Chips Shop,Theater
12,"Highland Creek,Rouge Hill,Port Union",0,Hotel,Italian Restaurant,Neighborhood,Burger Joint,Park,Grocery Store,Gym,Breakfast Spot,Deli / Bodega,Department Store
14,Woodbine Heights,0,Coffee Shop,Thai Restaurant,Pizza Place,Pharmacy,Gastropub,Sandwich Place,Park,Bar,Bank,Ice Cream Shop
15,St. James Town,0,Café,Coffee Shop,Restaurant,Hotel,Gastropub,Steakhouse,Cosmetics Shop,Italian Restaurant,American Restaurant,Seafood Restaurant
19,The Beaches,0,Coffee Shop,Pub,Breakfast Spot,Pizza Place,Beach,Grocery Store,Ice Cream Shop,Sandwich Place,Bar,Bakery
20,Berczy Park,0,Café,Coffee Shop,Hotel,Park,Restaurant,Japanese Restaurant,Seafood Restaurant,Bakery,Gastropub,Farmers Market
21,Caledonia-Fairbanks,0,Coffee Shop,Pizza Place,Italian Restaurant,Furniture / Home Store,Café,Bakery,Pharmacy,Grocery Store,Mexican Restaurant,Sandwich Place


### Cluster 2

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Lawrence Heights,Lawrence Manor",1,Clothing Store,Coffee Shop,Fast Food Restaurant,Sandwich Place,Cosmetics Shop,Furniture / Home Store,Dessert Shop,Vietnamese Restaurant,Bakery,Sporting Goods Shop
23,Leaside,1,Coffee Shop,Indian Restaurant,Bakery,Grocery Store,Restaurant,Burger Joint,Supermarket,Sporting Goods Shop,Park,Gym
25,Christie,1,Café,Coffee Shop,Korean Restaurant,Grocery Store,Bar,Indian Restaurant,Pizza Place,Ice Cream Shop,Cocktail Bar,Park
31,"Dovercourt Village,Dufferin",1,Café,Italian Restaurant,Bakery,Bar,Coffee Shop,Breakfast Spot,Park,Mexican Restaurant,Sushi Restaurant,Thai Restaurant
36,"Harbourfront East,Toronto Islands,Union Station",1,Hotel,Café,Coffee Shop,Restaurant,Italian Restaurant,Park,Gym / Fitness Center,Theater,Brewery,Sporting Goods Shop
37,"Little Portugal,Trinity",1,Café,Bar,Pizza Place,Restaurant,Bakery,Cocktail Bar,Coffee Shop,Italian Restaurant,Asian Restaurant,Theater
39,Bayview Village,1,Bank,Trail,Grocery Store,Japanese Restaurant,Park,Fast Food Restaurant,Café,Chinese Restaurant,Shopping Mall,Electronics Store
46,Downsview West,1,Moving Target,Bank,Plaza,Tea Room,Pizza Place,Park,Vietnamese Restaurant,Grocery Store,Shopping Mall,Zoo Exhibit
47,"The Beaches West,India Bazaar",1,Coffee Shop,Park,Café,Indian Restaurant,Pub,Bakery,Beach,Brewery,BBQ Joint,Restaurant
59,Willowdale South,1,Korean Restaurant,Japanese Restaurant,Bubble Tea Shop,Coffee Shop,Café,Pizza Place,Grocery Store,Ramen Restaurant,Sushi Restaurant,Restaurant


### Cluster 3

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Queen's Park,2,Coffee Shop,Japanese Restaurant,Gastropub,Park,Tea Room,Ramen Restaurant,Pizza Place,Italian Restaurant,Bookstore,Bubble Tea Shop
5,Islington Avenue,2,Shopping Mall,Pharmacy,Park,Café,Grocery Store,Bus Stop,Camera Store,Bus Line,Supermarket,Golf Course
7,Don Mills North,2,Japanese Restaurant,Coffee Shop,Restaurant,Burger Joint,Park,Pizza Place,Bank,Supermarket,Asian Restaurant,Café
10,Glencairn,2,Fast Food Restaurant,Coffee Shop,Grocery Store,Pizza Place,Bank,Sandwich Place,Pharmacy,Caribbean Restaurant,Fried Chicken Joint,Dessert Shop
13,"Flemingdon Park,Don Mills South",2,Coffee Shop,Japanese Restaurant,Gym,Sandwich Place,Restaurant,Middle Eastern Restaurant,Asian Restaurant,Grocery Store,Fast Food Restaurant,Beer Store
16,Humewood-Cedarvale,2,Coffee Shop,Italian Restaurant,Pizza Place,Café,Bank,Bakery,Indian Restaurant,Pharmacy,Caribbean Restaurant,Sandwich Place
17,"Bloordale Gardens,Eringate,Markland Wood,Old B...",2,Baseball Field,Convenience Store,Bank,Pizza Place,Coffee Shop,Grocery Store,Sandwich Place,Liquor Store,Garden,Beer Store
18,"Guildwood,Morningside,West Hill",2,Pizza Place,Fast Food Restaurant,Coffee Shop,Smoothie Shop,Asian Restaurant,Beer Store,Sandwich Place,Salon / Barbershop,Supermarket,Bank
26,Cedarbrae,2,Coffee Shop,Sandwich Place,Pharmacy,Clothing Store,Indian Restaurant,Bakery,Fast Food Restaurant,Bus Station,Wings Joint,Hotel
29,Thorncliffe Park,2,Coffee Shop,Sandwich Place,Restaurant,Grocery Store,Pharmacy,Electronics Store,Burger Joint,Indian Restaurant,Sporting Goods Shop,Turkish Restaurant


### Cluster 4

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,"Alderwood,Long Branch",3,Burger Joint,Toy / Game Store,Pizza Place,Pharmacy,Café,Grocery Store,Coffee Shop,Park,Beer Store,Bank


### Cluster 5

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,4,Coffee Shop,Middle Eastern Restaurant,Fast Food Restaurant,Gym,Grocery Store,Portuguese Restaurant,Cosmetics Shop,Beer Store,Spa,Soccer Field
2,"Harbourfront,Regent Park",4,Coffee Shop,Café,Breakfast Spot,Italian Restaurant,Restaurant,Park,Theater,Bakery,Bar,Farmers Market
6,"Rouge,Malvern",4,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Department Store,Cosmetics Shop,Fruit & Vegetable Store,Bus Station,Liquor Store,Spa,Paper / Office Supplies Store
24,Central Bay Street,4,Café,Ramen Restaurant,Coffee Shop,Hotel,American Restaurant,Breakfast Spot,Gastropub,Japanese Restaurant,Park,Bubble Tea Shop
53,Downsview Central,4,Vietnamese Restaurant,Pharmacy,Coffee Shop,Supermarket,Chinese Restaurant,Thrift / Vintage Store,BBQ Joint,Bank,Sandwich Place,New American Restaurant
61,Lawrence Park,4,Coffee Shop,Bakery,Italian Restaurant,Sushi Restaurant,Café,Asian Restaurant,Sandwich Place,Gym / Fitness Center,Burger Joint,Bank
